## 1. Informações

Este notebook implementa um sistema de RAG (Retrieval-Augmented Generation) utilizando o modelo Google Gemini 2.5 Flash para responder perguntas baseadas em uma base de conhecimento específica do ONS (Operador Nacional do Sistema Elétrico).

## 2. Importação de Bibliotecas

In [1]:
import warnings
from pathlib import Path
from typing import List, Dict

# Google Gemini
import google.generativeai as genai

# Processamento de PDFs
from PyPDF2 import PdfReader

# LangChain
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document

warnings.filterwarnings('ignore')

print("✓ Bibliotecas importadas com sucesso!")

c:\Users\mathe\miniconda3\envs\ia-ons\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



✓ Bibliotecas importadas com sucesso!
✓ Bibliotecas importadas com sucesso!


## 3. Configuração do Gemini API

In [ ]:
# Configurar API Key do Gemini
# Obtenha sua chave gratuita em: https://makersuite.google.com/app/apikey
GEMINI_API_KEY = " "  # Substitua pela sua chave

# Configurar Gemini
genai.configure(api_key=GEMINI_API_KEY)

# Usar modelo gratuito gemini-1.5-flash
model = genai.GenerativeModel('gemini-2.5-flash')

## 4. Carregamento e Processamento dos PDFs

### 4.1 Função para Extrair Texto dos PDFs

In [3]:
def extract_text_from_pdf(pdf_path: str) -> str:
    """
    Extrai texto de um arquivo PDF
    
    Args:
        pdf_path: Caminho para o arquivo PDF
    
    Returns:
        Texto extraído do PDF
    """
    try:
        reader = PdfReader(pdf_path)
        text = ""
        
        for page in reader.pages:
            text += page.extract_text() + "\n"
        
        return text.strip()
    
    except Exception as e:
        print(f"Erro ao processar {pdf_path}: {e}")
        return ""

### 4.2 Carregar Todos os PDFs

In [4]:
# Diretório dos PDFs
PDF_DIR = "./db/rag_context"

# Listar todos os PDFs
pdf_files = list(Path(PDF_DIR).glob("*.pdf"))

print("="*80)
print("PROCEDIMENTOS DE REDE DO ONS ENCONTRADOS")
print("="*80)
print(f"Total de arquivos PDF: {len(pdf_files)}\n")

for pdf_file in sorted(pdf_files):
    print(f"  • {pdf_file.name}")

# Carregar documentos
documents = []

print("\n" + "="*80)
print("PROCESSANDO PDFs...")
print("="*80)

for pdf_file in pdf_files:
    print(f"Processando: {pdf_file.name}...", end=" ")
    
    text = extract_text_from_pdf(str(pdf_file))
    
    if text:
        # Criar documento com metadados
        doc = Document(
            page_content=text,
            metadata={
                "source": pdf_file.name,
                "path": str(pdf_file)
            }
        )
        documents.append(doc)
        print(f"✓ ({len(text):,} caracteres)")
    else:
        print("✗ Falhou")

print(f"\n✓ {len(documents)} documentos carregados com sucesso!")

PROCEDIMENTOS DE REDE DO ONS ENCONTRADOS
Total de arquivos PDF: 16

  • submodulo_4.1_op.pdf
  • submodulo_4.1_resp.pdf
  • submodulo_4.2_proc.pdf
  • submodulo_4.2_resp.pdf
  • submodulo_4.3_proc.pdf
  • submodulo_4.3_resp.pdf
  • submodulo_4.4_op.pdf
  • submodulo_4.4_resp.pdf
  • submodulo_4.5_PDO_op.pdf
  • submodulo_4.5_PDO_resp.pdf
  • submodulo_4.6_proc.pdf
  • submodulo_4.6_resp.pdf
  • submodulo_4.7_op.pdf
  • submodulo_4.7_resp.pdf
  • submodulo_4.8_op.pdf
  • submodulo_4.8_resp.pdf

PROCESSANDO PDFs...
Processando: submodulo_4.1_op.pdf... ✓ (15,365 caracteres)
Processando: submodulo_4.1_resp.pdf... ✓ (10,697 caracteres)
Processando: submodulo_4.2_proc.pdf... ✓ (10,697 caracteres)
Processando: submodulo_4.2_proc.pdf... ✓ (42,870 caracteres)
Processando: submodulo_4.2_resp.pdf... ✓ (42,870 caracteres)
Processando: submodulo_4.2_resp.pdf... ✓ (28,182 caracteres)
Processando: submodulo_4.3_proc.pdf... ✓ (28,182 caracteres)
Processando: submodulo_4.3_proc.pdf... ✓ (23,059 caracte

## 5. Criação do Vector Store (Chunking e Embeddings)

### 5.1 Dividir Documentos em Chunks

In [5]:
# Configurar text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,        # Tamanho de cada chunk
    chunk_overlap=200,      # Sobreposição entre chunks
    length_function=len,
    separators=["\n\n", "\n", ".", " ", ""]
)

print("="*80)
print("DIVIDINDO DOCUMENTOS EM CHUNKS")
print("="*80)
print(f"Chunk size: 1000 caracteres")
print(f"Chunk overlap: 200 caracteres\n")

# Dividir documentos
chunks = text_splitter.split_documents(documents)

print(f"✓ {len(chunks)} chunks criados")
print(f"\nEstatísticas:")
print(f"  • Documentos originais: {len(documents)}")
print(f"  • Chunks totais: {len(chunks)}")
print(f"  • Média de chunks por documento: {len(chunks)/len(documents):.1f}")

DIVIDINDO DOCUMENTOS EM CHUNKS
Chunk size: 1000 caracteres
Chunk overlap: 200 caracteres

✓ 349 chunks criados

Estatísticas:
  • Documentos originais: 16
  • Chunks totais: 349
  • Média de chunks por documento: 21.8


### 5.2 Criar Embeddings e Vector Store

In [6]:
print("="*80)
print("CRIANDO EMBEDDINGS E VECTOR STORE")
print("="*80)
print("Modelo de embeddings: all-MiniLM-L6-v2 (multilingual)")
print("Vector store: ChromaDB\n")

# Configuração de embeddings (modelo gratuito e multilíngue)
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={'device': 'cpu'}
)

print("Gerando embeddings e criando índice... (pode levar alguns minutos)")

# Criar vector store
vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory="./db/chroma_db"
)

print("\n✓ Vector store criado com sucesso!")
print(f"✓ Índice persistido em: ./db/chroma_db")

CRIANDO EMBEDDINGS E VECTOR STORE
Modelo de embeddings: all-MiniLM-L6-v2 (multilingual)
Vector store: ChromaDB

Gerando embeddings e criando índice... (pode levar alguns minutos)
Gerando embeddings e criando índice... (pode levar alguns minutos)

✓ Vector store criado com sucesso!
✓ Índice persistido em: ./db/chroma_db

✓ Vector store criado com sucesso!
✓ Índice persistido em: ./db/chroma_db


## 6. Sistema RAG - Retrieval e Generation

### 6.1 Função de Busca Semântica

In [7]:
def retrieve_relevant_docs(query: str, k: int = 5) -> List[Document]:
    """
    Recupera documentos relevantes do vector store
    
    Args:
        query: Pergunta do usuário
        k: Número de documentos a retornar
    
    Returns:
        Lista de documentos relevantes
    """
    # Buscar documentos similares
    docs = vectorstore.similarity_search(query, k=k)
    
    return docs


### 6.2 Função RAG Completa

In [12]:
def rag_query(question: str, k: int = 5, verbose: bool = True) -> Dict:
    """
    Sistema RAG completo: Retrieval + Generation
    
    Args:
        question: Pergunta do usuário
        k: Número de documentos a recuperar
        verbose: Mostrar documentos recuperados
    
    Returns:
        Dicionário com resposta e metadados
    """
    
    if verbose:
        print("="*80)
        print("SISTEMA RAG - PROCESSANDO CONSULTA")
        print("="*80)
        print(f"Pergunta: {question}\n")
    
    # 1. RETRIEVAL: Buscar documentos relevantes
    if verbose:
        print(f"[1/3] Buscando {k} documentos relevantes...")
    
    relevant_docs = retrieve_relevant_docs(question, k=k)
    
    if verbose:
        print(f"✓ {len(relevant_docs)} documentos recuperados\n")
        print("Documentos recuperados:")
        for i, doc in enumerate(relevant_docs, 1):
            print(f"  {i}. {doc.metadata['source']} ({len(doc.page_content)} chars)")
    
    # 2. CONTEXTO: Concatenar documentos
    if verbose:
        print(f"\n[2/3] Preparando contexto...")
    
    context = "\n\n".join([
        f"[Documento: {doc.metadata['source']}]\n{doc.page_content}"
        for doc in relevant_docs
    ])
    
    if verbose:
        print(f"✓ Contexto preparado ({len(context):,} caracteres)")
    
    # 3. GENERATION: Gerar resposta com Gemini
    if verbose:
        print(f"\n[3/3] Gerando resposta com Gemini...")
    
    # Criar prompt para o Gemini
    prompt = f"""Você é um assistente especializado em Procedimentos de Rede do ONS (Operador Nacional do Sistema Elétrico).

Use APENAS as informações do contexto abaixo para responder à pergunta. Se a informação não estiver no contexto, diga que não encontrou a informação nos documentos fornecidos.

CONTEXTO:
{context}

PERGUNTA: {question}

INSTRUÇÕES:
1. Responda em português do Brasil
2. Seja claro e objetivo
3. Cite o documento fonte quando relevante (exemplo: "Segundo o submódulo 4.1...")
4. Se houver múltiplas informações relevantes, organize em tópicos
5. Não invente informações que não estão no contexto

RESPOSTA:"""
    
    # Gerar resposta
    response = model.generate_content(prompt)
    
    if verbose:
        print("✓ Resposta gerada\n")
        print("="*80)
        print("RESPOSTA")
        print("="*80)
        print(response.text)
    
    # Retornar resultado
    result = {
        "question": question,
        "answer": response.text,
        "sources": [doc.metadata['source'] for doc in relevant_docs],
        "num_docs": len(relevant_docs)
    }
    
    return result


## 7. Exemplos de Uso

In [14]:
# Fazer uma pergunta sobre os procedimentos
result = rag_query(
    question="O que é feito na Programação Diária da Operação (PDO)?",
    k=3,
    verbose=True
)

SISTEMA RAG - PROCESSANDO CONSULTA
Pergunta: O que é feito na Programação Diária da Operação (PDO)?

[1/3] Buscando 3 documentos relevantes...
✓ 3 documentos recuperados

Documentos recuperados:
  1. submodulo_4.5_PDO_resp.pdf (970 chars)
  2. submodulo_4.5_PDO_resp.pdf (970 chars)
  3. submodulo_4.5_PDO_op.pdf (982 chars)

[2/3] Preparando contexto...
✓ Contexto preparado (3,044 caracteres)

[3/3] Gerando resposta com Gemini...
✓ Resposta gerada

RESPOSTA
Na Programação Diária da Operação (PDO) são realizadas as seguintes atividades e apresentadas as seguintes informações:

*   **Consolidação e Fornecimento de Diretrizes:**
    *   Consolida as propostas de geração hidráulica, térmica, eólica e solar como a programação diária eletroenergética.
    *   Fornece aos centros de operação do ONS e aos agentes de operação as diretrizes eletroenergéticas específicas, necessárias à execução da operação. (Segundo o documento submodulo_4.5_PDO_resp.pdf, item 2.1.1)

*   **Composição e Conteúdo:*

In [15]:
# Pergunta mais técnica
result = rag_query(
    question="Quais são as responsabilidades dos agentes no planejamento da operação?",
    k=5,
    verbose=True
)

SISTEMA RAG - PROCESSANDO CONSULTA
Pergunta: Quais são as responsabilidades dos agentes no planejamento da operação?

[1/3] Buscando 5 documentos relevantes...
✓ 5 documentos recuperados

Documentos recuperados:
  1. submodulo_4.1_resp.pdf (943 chars)
  2. submodulo_4.1_resp.pdf (943 chars)
  3. submodulo_4.5_PDO_op.pdf (938 chars)
  4. submodulo_4.5_PDO_op.pdf (938 chars)
  5. submodulo_4.1_op.pdf (986 chars)

[2/3] Preparando contexto...
✓ Contexto preparado (4,938 caracteres)

[3/3] Gerando resposta com Gemini...
✓ Resposta gerada

RESPOSTA
Com base nos documentos fornecidos, as responsabilidades dos agentes no planejamento da operação são:

*   **Informar dados:** Os agentes de operação informam dados que são posteriormente verificados pelo ONS para eventuais inconsistências (submódulo 4.1_resp.pdf).
*   **Solicitar inclusão de análises no escopo:** Os agentes de operação podem solicitar a inclusão, no escopo dos estudos para o horizonte mensal, de análises que considerem pertinent

In [17]:
# Pergunta mais técnica
result = rag_query(
    question="Quais são as responsabilidades do ONS na Programação Diária da Operação?",
    k=5,
    verbose=True
)

SISTEMA RAG - PROCESSANDO CONSULTA
Pergunta: Quais são as responsabilidades do ONS na Programação Diária da Operação?

[1/3] Buscando 5 documentos relevantes...
✓ 5 documentos recuperados

Documentos recuperados:
  1. submodulo_4.5_PDO_op.pdf (938 chars)
  2. submodulo_4.5_PDO_op.pdf (938 chars)
  3. submodulo_4.2_resp.pdf (978 chars)
  4. submodulo_4.2_resp.pdf (978 chars)
  5. submodulo_4.2_resp.pdf (985 chars)

[2/3] Preparando contexto...
✓ Contexto preparado (5,009 caracteres)

[3/3] Gerando resposta com Gemini...
✓ Resposta gerada

RESPOSTA
Segundo o Submódulo 4.5 – PDO Op., as responsabilidades do ONS na Programação Diária da Operação são:

*   **Assegurar Transparência e Interatividade:** O processo de programação diária da operação eletroenergética é participativo e interativo, com reprodutibilidade de resultados e transparência entre o ONS e os agentes com responsabilidades definidas neste submódulo (item 1.4).
*   **Utilizar Critérios Definidos:** Para a elaboração do progra

In [16]:
# Pergunta sobre procedimento específico
result = rag_query(
    question="Como funciona o processo de programação da operação elétrica?",
    k=4,
    verbose=True
)

SISTEMA RAG - PROCESSANDO CONSULTA
Pergunta: Como funciona o processo de programação da operação elétrica?

[1/3] Buscando 4 documentos relevantes...
✓ 4 documentos recuperados

Documentos recuperados:
  1. submodulo_4.5_PDO_op.pdf (938 chars)
  2. submodulo_4.5_PDO_op.pdf (938 chars)
  3. submodulo_4.2_resp.pdf (985 chars)
  4. submodulo_4.2_resp.pdf (985 chars)

[2/3] Preparando contexto...
✓ Contexto preparado (4,000 caracteres)

[3/3] Gerando resposta com Gemini...
✓ Resposta gerada

RESPOSTA
De acordo com os documentos fornecidos, o processo de programação da operação elétrica, especificamente a programação diária da operação eletroenergética, funciona da seguinte forma:

*   **Natureza do Processo:** É um processo participativo e interativo, que garante a reprodutibilidade de resultados e a transparência entre o ONS e os agentes com responsabilidades definidas no submódulo, os quais devem estar capacitados em todas as suas etapas (submodulo_4.5_PDO_op.pdf, item 1.4).
*   **Elabor

## 8. Análise e Estatísticas

### 8.1 Estatísticas do Vector Store

In [21]:
print("="*80)
print("ESTATÍSTICAS DO SISTEMA RAG")
print("="*80)

print(f"\n📁 Documentos:")
print(f"  • PDFs processados: {len(pdf_files)}")
print(f"  • Documentos carregados: {len(documents)}")
print(f"  • Chunks criados: {len(chunks)}")

print(f"\n🔍 Vector Store:")
print(f"  • Modelo de embeddings: all-MiniLM-L6-v2")
print(f"  • Dimensão dos vetores: 384")
print(f"  • Database: ChromaDB")

print(f"\n🤖 Modelo LLM:")
print(f"  • Modelo: Gemini 2.5 Flash")
print(f"  • Tier: Free")
print(f"  • Provider: Google")

# Listar todos os documentos
print(f"\n📄 Documentos disponíveis:")
for i, pdf_file in enumerate(sorted(pdf_files), 1):
    print(f"  {i}. {pdf_file.name}")

ESTATÍSTICAS DO SISTEMA RAG

📁 Documentos:
  • PDFs processados: 16
  • Documentos carregados: 16
  • Chunks criados: 349

🔍 Vector Store:
  • Modelo de embeddings: all-MiniLM-L6-v2
  • Dimensão dos vetores: 384
  • Database: ChromaDB

🤖 Modelo LLM:
  • Modelo: Gemini 2.5 Flash
  • Tier: Free
  • Provider: Google

📄 Documentos disponíveis:
  1. submodulo_4.1_op.pdf
  2. submodulo_4.1_resp.pdf
  3. submodulo_4.2_proc.pdf
  4. submodulo_4.2_resp.pdf
  5. submodulo_4.3_proc.pdf
  6. submodulo_4.3_resp.pdf
  7. submodulo_4.4_op.pdf
  8. submodulo_4.4_resp.pdf
  9. submodulo_4.5_PDO_op.pdf
  10. submodulo_4.5_PDO_resp.pdf
  11. submodulo_4.6_proc.pdf
  12. submodulo_4.6_resp.pdf
  13. submodulo_4.7_op.pdf
  14. submodulo_4.7_resp.pdf
  15. submodulo_4.8_op.pdf
  16. submodulo_4.8_resp.pdf


### 8.2 Teste de Busca Semântica

In [22]:
# Testar busca semântica direta
test_query = "planejamento da operação"

print("="*80)
print(f"TESTE DE BUSCA SEMÂNTICA")
print("="*80)
print(f"Query: '{test_query}'\n")

results = retrieve_relevant_docs(test_query, k=5)

print(f"Top {len(results)} documentos mais relevantes:\n")

for i, doc in enumerate(results, 1):
    print(f"{i}. {doc.metadata['source']}")
    print(f"   Preview: {doc.page_content[:150].replace(chr(10), ' ')}...")
    print()

TESTE DE BUSCA SEMÂNTICA
Query: 'planejamento da operação'

Top 5 documentos mais relevantes:

1. submodulo_4.1_resp.pdf
   Preview: Planejamento da operação elétrica com horizonte mensal para a configuração da Rede de Operação. 2. PRODUTO 2.1. Diretrizes para Operação Elétrica com ...

2. submodulo_4.1_resp.pdf
   Preview: Planejamento da operação elétrica com horizonte mensal para a configuração da Rede de Operação. 2. PRODUTO 2.1. Diretrizes para Operação Elétrica com ...

3. submodulo_4.1_op.pdf
   Preview: planejamento da operação elétrica de médio prazo e o relatório de Diretrizes para Operação Elétrica  com Horizonte Quadrimestral para abordar, por exe...

4. submodulo_4.1_op.pdf
   Preview: planejamento da operação elétrica de médio prazo e o relatório de Diretrizes para Operação Elétrica  com Horizonte Quadrimestral para abordar, por exe...

5. submodulo_4.2_proc.pdf
   Preview: apenas intervenções com desligamento de equipamentos principais dessas instalações, conforme a  cla

## 9. Conclusões

### **Sistema RAG Implementado:**

1. **Processamento de Documentos**:
   - Extração de texto de 16 PDFs de procedimentos do ONS
   - Divisão em chunks de 1000 caracteres com overlap de 200
   - Total de chunks indexados no vector store

2. **Retrieval (Busca)**:
   - Embeddings com modelo multilíngue all-MiniLM-L6-v2
   - Vector store ChromaDB para busca semântica eficiente
   - Recuperação dos top-k documentos mais relevantes

3. **Generation (Geração)**:
   - Integração com Google Gemini 1.5 Flash (free tier)
   - Prompts otimizados para respostas precisas e contextualizadas
   - Citação automática de fontes